# Export Alexnet from Torchvision Models
In this notebook we convert Alexnet to ONNX and upload it to S3 where it can be deployed by Cortex

Based on: [PytorchOnnxExport](https://github.com/onnx/tutorials/blob/master/tutorials/PytorchOnnxExport.ipynb)

## Install dependencies

In [0]:
!pip install torch==1.2.* torchvision==0.4.*

## Download and Export Model
Download the pretrained Alexnet Model and export to ONNX model format:

In [0]:
import torch
import torch.onnx
import torchvision

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter since we only care about network structure.
dummy_input = torch.randn(1, 3, 224, 224)

# We are going to use a Pretrained alexnet model
model = torchvision.models.alexnet(pretrained=True)

# Export to ONNX
torch.onnx.export(model, dummy_input, "alexnet.onnx")

## Upload the model to AWS
Cortex loads models from AWS, so we need to upload the exported model.

Set these variables to configure your AWS credentials and model upload path:

In [0]:
AWS_ACCESS_KEY_ID = "" #@param {type:"string"}
AWS_SECRET_ACCESS_KEY = "" #@param {type:"string"}
S3_UPLOAD_PATH = "s3://my-bucket/image-classifier/alexnet.onnx" #@param {type:"string"}

import sys
import re

if AWS_ACCESS_KEY_ID == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_ACCESS_KEY_ID"), file=sys.stderr)

elif AWS_SECRET_ACCESS_KEY == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_SECRET_ACCESS_KEY"), file=sys.stderr)

else:
    try:
        bucket = re.search("s3://(.+?)/", S3_UPLOAD_PATH).group(1)
        key = re.search("s3://.+?/(.+)", S3_UPLOAD_PATH).group(1)
    except:
        print("\033[91m{}\033[00m".format("ERROR: Invalid s3 path (should be of the form s3://my-bucket/path/to/file)"), file=sys.stderr)

Upload the model to S3:

In [0]:
import boto3

s3 = boto3.client("s3", aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
print("Uploading {} ...".format(S3_UPLOAD_PATH), end = '')
s3.upload_file("alexnet.onnx", bucket, key)
print(" ✓")

<!-- CORTEX_VERSION_MINOR -->
That's it! See the [example on GitHub](https://github.com/cortexlabs/cortex/tree/0.9/examples/image-classifier) for how to deploy the model as an API.